In [10]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

from nltk.corpus import brown

In [8]:
nltk.download('brown')

[nltk_data] Downloading package brown to /home/ashish/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [9]:
print(brown.tagged_words())

[('The', 'AT'), ('Fulton', 'NP-TL'), ...]


In [21]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/ashish/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [26]:
# storing all the POS tags, words
tags = defaultdict()
words = defaultdict()
tag_words = defaultdict()
for tuple in brown.tagged_words(tagset='universal'):
    tag = tuple[1]
    word = tuple[0]
    if tag in tags: tags[tag] += 1
    else: tags[tag] = 1
        
    if word in words: words[word] += 1
    else: words[word] = 1
        
    if tag in tag_words:
        if word in tag_words[tag]: tag_words[tag][word] += 1
        else: tag_words[tag][word] = 1
    else:
        tag_words[tag] = defaultdict()
        tag_words[tag][word] = 1

In [35]:
# checking tag_words
word = "this"
tag = "DET"
if word in tag_words[tag]:
    print(tag_words[tag][word])

3965


In [23]:
# checking tags
for key,value in tags.items():
    print(key, value)

DET 137019
NOUN 275558
ADJ 83721
VERB 182750
ADP 144766
. 147565
ADV 56239
CONJ 38151
PRT 29829
PRON 49334
NUM 14874
X 1386


In [24]:
for key,value in words.items():
    print(key, value)

The 7258
Fulton 17
County 85
Grand 18
Jury 4
said 1943
Friday 60
an 3542
investigation 43
of 36080
Atlanta's 4
recent 167
primary 93
election 72
produced 90
`` 8837
no 1781
evidence 201
'' 8789
that 10237
any 1301
irregularities 8
took 425
place 528
. 49346
jury 63
further 194
in 19536
term-end 1
presentments 1
the 62713
City 134
Executive 9
Committee 88
, 58334
which 3540
had 5102
over-all 35
charge 120
deserves 16
praise 17
and 27915
thanks 27
Atlanta 35
for 8841
manner 124
was 9777
conducted 55
September-October 1
term 79
been 2470
charged 57
by 5103
Superior 16
Court 110
Judge 39
Durwood 1
Pye 1
to 25732
investigate 11
reports 78
possible 373
hard-fought 2
won 68
Mayor-nominate 1
Ivan 4
Allen 20
Jr. 75
Only 102
a 21881
relative 44
handful 13
such 1192
received 163
considering 39
widespread 30
interest 323
number 467
voters 18
size 138
this 3966
city 259
it 6723
did 994
find 395
many 925
Georgia's 9
registration 23
laws 87
are 4333
outmoded 4
or 4118
inadequate 32
often 349
ambiguou

Weatherford 2
insisted 43
development 310
Representatives 14
effort 145
Fort 47
Worth 3
cover 85
places 99
Wise 3
hamlet 3
250 11
shouting 32
ended 59
114 4
sending 34
sponsored 30
Most 104
$5,000,000 3
$15,000,000 2
maximum 78
loan 41
could 1580
project 91
construed 6
large 354
felt 357
spent 104
providing 56
Statements 1
paying 26
benefit 62
pending 13
sway 5
Cotten's 1
attack 103
bill's 2
defenders 6
mostly 38
small-town 6
Buchanan 2
Dumas 1
Eligio 1
Kika 1
De 74
La 42
Garza 1
Mission 16
F. 59
Collins 6
Newton 6
Chapman 6
Sulphur 2
Springs 13
poor 92
boy's 16
peanut 5
serving 37
reducing 30
learning 60
educational 65
methods 138
C. 93
Grover 4
teaches 11
history 268
24 48
12 96
semester 10
hours 173
so-called 31
teaching 61
required 181
junior 29
senior 31
high 461
normal 135
college 163
30 112
junior-senior 1
teachers 64
least 343
credit 55
subject 158
remainder 19
4-year 2
subjects 80
person 170
master's 6
degree 125
physics 19
chemistry 16
math 3
English 195
permitted 57
teach 41

copy 38
released 26
responding 6
urges 8
complete 179
eye 119
legislature 12
Legislative 1
perform 29
review 46
Atty. 4
Gen. 23
Joseph 55
Nugent 5
views 51
appoint 6
Nugent's 1
expect 107
religious 158
labor 119
special-interest 1
affected 36
wrote 181
continuous 41
confronts 5
enforcement 19
officers 81
regulating 4
advised 33
enforce 8
Should 23
shirking 2
activity 115
across 274
statutes 7
Bay 38
dating 4
instances 30
colonial 14
times 264
severely 16
limit 47
types 114
merchandise 6
sold 47
Sabbath 2
concern 97
especially 156
foods 42
placed 126
list 130
neighborhood 58
grocery 7
variety 84
stores 34
chance 129
compete 23
supermarkets 3
council's 2
small 518
shops 16
retained 22
livelihood 5
thousands 44
declares 11
licenses 5
revenue 19
factory 23
outlets 6
operate 48
contended 12
shopping 26
Liberals 5
conservatives 4
parties 58
independent 66
Reama 5
nationally 10
known 245
labor-management 7
expert 30
Rotary 5
Club 76
luncheon 23
Sheraton-Biltmore 3
Hotel 43
type 197
enterprise

Lieberman 1
Eugene 25
Gillis 1
Petitions 1
Norristown 1
Julian 8
Barnard 5
Montgomery 16
Horace 5
Davenport 2
widow 23
killed 75
Barnard's 1
hit-run 2
Dannehower 1
pleaded 7
manslaughter 4
fined 4
Warren 50
K. 17
Hess 1
years' 8
probation 7
drive 93
driving 51
sentence 33
pronounced 18
Victim 1
accident 32
Lee 34
Stansbery 1
39 5
reprimanded 2
violating 4
Planning 14
agreements 15
redevelopers 1
Redevelopment 4
$300,000,000 1
Eastwick 5
Area 5
hazards 9
pedestrians 2
Corp. 11
1311 1
acre 10
tract 15
$12,192,865 1
bounded 9
Dicks 1
61st 1
parks 15
designated 17
Stage 2
Residential 4
Authority's 3
master 61
feature 36
row 34
garden 47
apartments 16
churches 82
clusters 5
corporation 70
formed 76
Reynolds 5
Metal 3
builder 26
second 341
520-acre 1
west 48
builders 25
developing 51
Would 37
bar 71
Constantinos 1
Doxiadis 1
Reconstruction 3
Greece 16
planner 2
sectors 10
barred 8
vehicular 1
landscaped 3
walkways 1
esplanade 3
entire 149
two-and-a-half-mile 1
length 115
eliminates 4
Grovers

Hinton 3
27 26
innings 4
unusual 62
characteristic 68
lagged 2
McAuliffe 4
cracked 17
Lefty 2
Don 22
Rudolph 3
Bear's 1
Paschal 2
gruonded 1
Jay 14
Cooke 3
McDaniel 2
Alusik 1
Porter 11
bases 23
Wert's 1
smash 4
putout 1
leftfield 1
rightfield 3
Phil 65
Shartzer's 1
3-hitter 1
Indians 32
bunched 5
Chuck 6
tripled 4
Cliff 3
Dan 27
Pavletich 1
Gaines' 1
infield 7
roller 3
accounted 5
Alusik's 2
outfield 3
Wert 2
Gaines 2
hammered 3
45-degree 3
clicked 8
1:48 2
Chico 2
Ruiz 1
spectacular 21
grounder 1
showed 141
arm 91
Bingles 1
bobbles 1
Tribe's 1
season's 3
obvious 92
refocusing 1
lights 46
flooded 9
dark 170
Dobbs 1
organ 11
exotic 7
dancer 31
Patti 1
Waggin 1
Wyman 1
Tsitouris 1
o'clock 39
Donnelly 1
Kenny 2
Lane 19
Muskegon 1
Mich. 4
world's 32
ranked 4
lightweight 4
Rip 1
Randall 1
Paul-Minneapolis 1
Gardner's 4
eluded 2
diving 3
Minnie 3
Minoso 1
Lemon 4
Minnesota 13
Twins 5
6-5 1
liner 3
5777 1
fans 21
batted 2
Turk 5
Lown 1
tagged 6
Ray 16
Moore 24
Reno 7
Bertoia 1
chopper 1
Lenn

...

worryin' 1
spades 4
crooks 2
distractedly 1
Angelo's 2
regular-featured 1
spaniel-like 1
Prettyman 2
Denny 2
anybody'd 1
everything's 2
underling 1
Domokous 2
Skyros' 2
paternally 1
Elite 1
ground-glass 1
ten-twelve 1
Bouvardier 1
superstitious 1
Katya 3
Roslev 1
Katharine 6
'way 1
so's 4
there'd 3
foreign-sounding 1
Prettier 1
smarter 1
vaguely-imagined 1
Katharine's 1
Mendoza 1
Alison 1
handbag 3
propped 3
Bast 1
daintily 1
Abyssinians 1
kittens 5
que 1
sigue 1
despues 1
vet 1
Carters 1
Maude's 1
Happened 3
Lolotte 8
beau 1
Emile 5
Maude 21
Celie's 1
Sarah's 4
smokehouse 2
storehouse 3
Drink 3
hiked 1
Lucien 10
floes 1
Stowe 1
paterollers 1
swished 4
undrinkable 1
Honotassa 4
Rilly 1
Glendora 13
unprocurable 1
credible 2
unpleasantly 1
Rev 11
Emile's 2
Added 1
moire 1
armoire 1
moccasins 2
curtly 2
dosed 2
doled 1
Celie 1
storeroom 1
Commissary 1
hams 1
smoke-stained 1
rafters 1
ruthlessness 2
drizzling 2
brad 1
George's 3
Reckon 2
satirically 1
Rev's 2
cobbler's 1
lamplight 3
Lolott

decorticated 1
lovelies 1
Earth-weeks 1
Earth-week 1
well-read 1
dromozootic 1
implant 1
senselessly 1
hoarsely 4
child-face 1
dabbed 1
toe-tips 1
teratologies 1
nicest 1
burned-out 1
ground-truck 1
threshed 1
bawled 2
perplexity 1
eighty-four 1
cow-man 1
cow-people 1
resembling 2
kindliness 1
geysers 2
commingled 1
B'dikkat's 1
fickle 1
ditches 2
Renfro 1
angling 1
Reaching 4
Leaning 3
frowzy 1
dipper 6
greedily 1
nester 2
buckets 3
hell's 2
half-acre 1
Buckhorn's 1
woebegone 1
idiot 2
dejectedly 1
Rafter 1
best-looking 1
innocents 1
doc 1
scissors 1
savvy 1
Rittenhouse 1
Splendide 1
croaks 1
half-breed 5
'pache 1
anythin 1
goin 4
murmuring 4
fiercely 4
driftin 1
stallion 3
tightened 6
cinches 1
looped 1
veranda 8
somethin 1
stallion's 2
huskily 1
Lived 2
rump 2
campfire 2
untenanted 1
palely 1
gullies 1
watchful 2
windless 1
Pettigrew's 1
leavin 2
Purvis 4
rustled 2
Clayton's 3
Lester's 1
Cabot's 1
Silas 1
Pettigrew 1
Thirty-six 1
manes 2
pennants 1
hoofs 7
mare's 1
leaping 2
Enfield

Scobee-Frazier 1
Expedition 1
Manitoba 1
Karipo 4
inter-tribal 1
rallied 1
Pamasu 1
feminist 1
betel-stained 1
Karipo's 1
squalid 1
semi-circle 1
squaw 1
Ramey's 3
bib 2
Ramey 28
dislodged 1
Nope 1
checker 1
far-away 1
floppy 1
shredded 1
joyfully 1
Horsely 2
ex-truck 1
outlandish 1
Crumley 1
towed 1
Slender 1
grimed 1
diesel 1
alleys 1
blonde-haired 1
courtliness 1
levi-clad 1
half-off 1
laces 1
eyelets 1
blonde-headed 1
Coca-Cola 1
Benson's 1
sneaker 2
nudge 2
blood-flecked 1
light-headedness 1
Them's 2
once-in-a-lifetime 1
cheekbone 1
moaned 2
kneeled 2
gesturing 1
ruddiness 1
patrolman's 1
smoky 1
horned 1
Mough 1
mough 1
bungalow 1
reciprocate 1
harsher 1
roach 1
Ahmiri 1
invader 1
sulking 1
Brahmaputra 2
tableland 1
headwalls 2
pinnacles 1
glaciers 1
rockstrewn 1
sky-carving 1
massifs 1
Gurla 1
Mandhata 1
Kemchenjunga 1
curling 1
Assam 2
Khasi 1
Sirinjani 1
Madaripur 1
mingles 1
meld 1
teeming 1
Hump 2
untracked 1
lords 1
Wing 1
Commanders 1
RAF 3
Keith's 3
crates 2
swastika 1
bo

bellicosity 1
disheveled 2
Appaloosas 1
multicolored 1
cacophony 1
thousand-legged 1
hemlocks 1
birch 1
short-cut 1
frogs 1
lower-cut 1
Bentley 2
Rawlings 4
bandages 3
coursing 1
blood-soaked 1
tidiness 1
Poldowski 1
effete 1
shoelaces 1
absurdly 1
gauze 1
detained 1
freakish 1
cavern 1
blazer 1
Pietro's 1
implausibly 1
Pietro 2
Ardmore 1
Carrie 1
purposeless 1
Abruptly 1
bumped 2
intoxicated 1
pinch-hit 1
tactful 2
bribe 1
flyaway 1
jollying 1
Upstairs 1
showering 1
raindrops 1
pattered 1
prudent 1
Websterville 2
Myra's 4
nervously 3
child-cloud 1
forever-Cathy 1
moon-washed 1
muddling 1
clattering 1
shape-up 1
Susan's 2
outsized 1
armload 1
Scrooge-like 1
cousins' 1
Cathy's 1
Lilliputian 1
flower-scented 1
good-by 3
chillier 1
raffish 1
cowbirds 3
vestments 1
Goose 2
hangers 1
panicky 1
napping 1
Subdued 1
merriest 1
voiceless 1
Puzzled 1
moth 1
seekingly 1
Christmastime 2
uncoiling 1
to-and-fro 1
hall's 1
purple-black 1
cowbirds' 1
Cowbird 1
teddy 2
aunt's 1
Unimpressed 1
plopped 1


lemon-meringue 1
muffins 1
Rilke 1
unpretentious 1
Aging 1
enfant 1
Francoisette 1
Lagoon 2
scenario 1
Lascivious 1
Interlude 1
trip-hammer 1
pithy 1
all-pervading 1
hollowness 1
Mlle 1
Petite 1
Chadroe 1
engagingly 1
Bambi 1
Fink's 1
understated 1
Quizzical 1
Salamander 1
salamander 1
Alicia 4
tragically 1
Parkinson's 1
mah-jongg 1
Fing 2
Pulova 2
Hardwicke 1
Nabisco 1
bout-de-souffle 1
leitmotiv 1
Mudugno 1
Volare 1
opalescent 1
fascinatingly 1
Rosalie 1
beat-up 1
jeunes 1
Cause 1
pug-nosed 1
Jean-Pierre 1
Bravado 2
Bogartian 1
Tasti-Freeze 1
New-Waver 1
Fredrico 1
Rossilini's 1
Sour 1
Sponge 1
crackles 1
Soaring 1
Margo 1
Felicity 1
Brighetti 1
curvaceously 1
drib-drool 1
sophisticates 1
Expressionism 1
Vindication 1
Quasimodo's 2
Guggenheim 1
Quasimodo 3
Kandinsky 1
blossomed 1
Kline 1
inexpressible 1
unpaintable 1
non-time 1
unwaivering 1
Invasion 1
non-color 1
chiding 1
jocose 1
reliably 1
argot 1
hot-slough 1
prowlers 1
taunting 1
etymological 1
rifling 1
whimper 1
objets 1
d'ar

## Probabilities to compute: 
    P(word|tag) = count(word,tag)/count(all words,tag) -> emission probabilities
    P(tag1|tag2) = count(tag2,tag1)/count(tag2,tagx) -> transition probabilities
 


## Emission Probabilities

In [46]:
# computing emission probabilities
emission = defaultdict()
for tag in tags.keys():
    for word in words.keys():
        
        if tag not in emission:
            emission[tag] = defaultdict()
        count = 0
        if word in tag_words[tag]: count = tag_words[tag][word]
        emission[tag][word] = count/tags[tag]

In [47]:
# checking probabilities
tag = "DET"
word = "the"
emission[tag][word]

0.45767375327509324

## Computing bi-gram counts

In [60]:
sents = []
for lis in brown.tagged_sents(tagset='universal'):
    sents.append(lis)

print(len(sents))

57340


In [73]:
for sent in sents:
    if len(sent) == 5: print(sent)

[('``', '.'), ('Must', 'VERB'), ('solve', 'VERB'), ('problem', 'NOUN'), ("''", '.')]
[('The', 'DET'), ('hotel', 'NOUN'), ('owner', 'NOUN'), ('shrugged', 'VERB'), ('.', '.')]
[('Formula', 'NOUN'), ('is', 'VERB'), ('due', 'ADJ'), ('this', 'DET'), ('week', 'NOUN')]
[('Oak', 'NOUN'), ('Grove', 'NOUN'), ('(', '.'), ('special', 'ADJ'), (')', '.')]
[('--', '.'), ('emphasizes', 'VERB'), ('the', 'DET'), ('Virgin', 'NOUN'), ('birth', 'NOUN')]
[("'", '.'), ('church', 'NOUN'), ('meets', 'VERB'), ('change', 'NOUN'), ("'", '.')]
[('Seeks', 'VERB'), ('``', '.'), ('improved', 'VERB'), ('fielding', 'VERB'), ("''", '.')]
[('Duren', 'NOUN'), (',', '.'), ('Sheldon', 'NOUN'), ('on', 'ADP'), ('hill', 'NOUN')]
[('A', 'DET'), ('quick', 'ADJ'), ('touchdown', 'NOUN'), ('resulted', 'VERB'), ('.', '.')]
[('It', 'PRON'), ('made', 'VERB'), ('him', 'PRON'), ('human', 'NOUN'), ('.', '.')]
[('He', 'PRON'), ('had', 'VERB'), ('a', 'DET'), ('16', 'NUM'), ('.', '.')]
[('The', 'DET'), ('ball', 'NOUN'), ('floated', 'VERB'),

[('He', 'PRON'), ('is', 'VERB'), ('innocent', 'ADJ'), ("''", '.'), ('.', '.')]
[('Rachel', 'NOUN'), ('talked', 'VERB'), ('to', 'ADP'), ('him', 'PRON'), ('.', '.')]
[('The', 'DET'), ('boys', 'NOUN'), ('were', 'VERB'), ('tiny', 'ADJ'), ('.', '.')]
[('She', 'PRON'), ('passed', 'VERB'), ('by', 'ADP'), ('him', 'PRON'), ('.', '.')]
[('I', 'PRON'), ('grinned', 'VERB'), ('at', 'ADP'), ('Rachel', 'NOUN'), ('.', '.')]
[('She', 'PRON'), ('smiled', 'VERB'), ('to', 'ADP'), ('herself', 'PRON'), ('.', '.')]
[('``', '.'), ('Here', 'ADV'), (',', '.'), ('nothing', 'NOUN'), ('.', '.')]
[('She', 'PRON'), ('loved', 'VERB'), ('the', 'DET'), ('children', 'NOUN'), ('.', '.')]
[('She', 'PRON'), ('worked', 'VERB'), ('very', 'ADV'), ('hard', 'ADV'), ('.', '.')]
[('she', 'PRON'), ('was', 'VERB'), ('terribly', 'ADV'), ('alone', 'ADV'), ('.', '.')]
[('she', 'PRON'), ('did', 'VERB'), ('not', 'ADV'), ('care', 'VERB'), ('.', '.')]
[('You', 'PRON'), ('could', 'VERB'), ('wish', 'VERB'), ('that', 'DET'), ('.', '.')]
[('n

In [61]:
bigram = defaultdict()
for sent in sents:
    length = len(sent)
    for i in range(0, length-1):
        curr = sent[i][1]
        next_ = sent[i+1][1]
        bi = curr + '-' + next_
        if bi not in bigram: bigram[bi] = 1
        else: bigram[bi] += 1

In [65]:
# test bi-grams
print(bigram["NOUN-DET"])
print(tags["NOUN"])

4270
275558


In [75]:
# compute bi-gram counts for starting tag and next tag
for sent in sents:
    start_tag = sent[0][1]
    bi = "^-" + start_tag
    if bi not in bigram: bigram[bi] = 1
    else: bigram[bi] += 1

## Transition Probabilities

In [76]:
# check bi-grams of states which will be divided by count of initial state
transition = defaultdict()
for tag in tags.keys():
    for tag1 in tags.keys():
        if tag not in transition:
            transition[tag] = defaultdict()
        transition[tag][tag1] = bigram[tag+'-'+tag1]/tags[tag] 

In [82]:
# compute starting tag transition probability i.e P(X|^)
transition["^"] = defaultdict()
for tag in tags.keys():
     transition["^"][tag] = bigram["^"+"-"+tag]/len(sents)

In [83]:
# test transition
print(transition["DET"]["NOUN"])
print(transition["^"]["ADJ"])

0.6264678621213117
0.06867806069061737


## Viterbi Algo

In [123]:
import copy

In [137]:
class TreeNode:
    def __init__(self, tag, prob, parent):
        self.tag = tag
        self.prob = prob
        self.tags = []
        self.parent = parent
        

class Viterbi:
    
    def find_tags(self, imp_nodes):
        max_node = None
        for tag,tag_node in imp_nodes.items():
            if max_node is None or max_node.prob < tag_node.prob: max_node = tag_node
        
        tags = []
        max_node = max_node.parent
        while(max_node is not None):
            tags.insert(0, max_node.tag)
            max_node = max_node.parent
        
        return tags
        
    def compute_states(self, sent):
        sent = sent.strip()
        tokens = sent.split(' ')
        if sent[-1] == ".":
            tokens[-1] = tokens[-1][:-1]
            tokens.append(".")
        
        root = TreeNode("^", 1, None)
        imp_nodes = defaultdict()
        
        # create first level of tree
        for tag in tags.keys():
            node = TreeNode(tag, transition["^"][tag], root)
            root.tags.append(copy.deepcopy(node))
            imp_nodes[tag] = copy.deepcopy(node)
        
        
        temp_best = defaultdict()
        for token in tokens:
            level_nodes = []
            temp_best = defaultdict()
            for tag,tag_node in imp_nodes.items():
                # compute every tag for this node
                for child_tag in tags.keys():
                    new_prob = tag_node.prob*emission[tag][token]*transition[tag][child_tag]
                    #print(token + " tag: " + tag + " child_tag: " + child_tag + " " + str(new_prob))
                    child = TreeNode(child_tag, new_prob, tag_node)
                    tag_node.tags.append(child)
                    level_nodes.append(child)
            
            # now select the best child for each tag
            for node in level_nodes:
                tag = node.tag
                #print(token + " " + tag + " " + str(node.prob))
                if tag in temp_best:
                    if node.prob > temp_best[tag].prob:
                        temp_best[tag] = node
                else:
                    temp_best[tag] = node
            
            for tag in tags.keys(): imp_nodes[tag] = copy.deepcopy(temp_best[tag])
        
        return self.find_tags(imp_nodes)
        
        

obj = Viterbi()
sent = "The hotel owner shrugged."
states = obj.compute_states(sent) 

In [136]:
states

['^', 'DET', 'NOUN', 'NOUN', 'VERB', '.']